In [1]:
from Launch import  Scheduler
import unittest
from datetime import datetime, timedelta, timezone



In [2]:
# როდესაც ჯობიისთის გადაცემული გაშვების თარიღი

In [ ]:

class TestScheduler(unittest.TestCase):

    def test_scheduler_initialization(self):
        """Test the initialization of the Scheduler class."""
        scheduler = Scheduler(threading=False, start_date="2024-12-25 10:00", time_zone=0)
        self.assertEqual(scheduler.scheduler_startdate.strftime('%Y-%m-%d %H:%M'), "2024-12-25 10:00")
        self.assertEqual(scheduler.scheduler_timezone, timezone(timedelta(hours=0)))

    def test_invalid_start_date(self):
        """Test that invalid start_date raises a ValueError."""
        with self.assertRaises(ValueError):
            Scheduler(start_date="invalid-date", time_zone=0)


    def test_job_creation(self):
        """Test creating a job within the scheduler."""
        timezonesc = 5
        timezonejob = 0
        local_timezone = datetime.now().astimezone().tzinfo


        start_datesc = "2024-12-25 10:00"
        startdatejob = "2024-12-25 11:00"
        scheduler = Scheduler(start_date=start_datesc, time_zone=timezonesc)
        job_to_run = scheduler.job(start_date=startdatejob, time_zone=timezonejob)

        if timezonejob:
            if local_timezone != timezonejob:
                start_date = datetime.strptime(startdatejob, '%Y-%m-%d %H:%M')
                comparison = start_date.astimezone(local_timezone)
                self.assertEqual(job_to_run.job['startdate'].strftime('%Y-%m-%d %H:%M'), comparison)
            else:
                self.assertEqual(job_to_run.job['startdate'].strftime('%Y-%m-%d %H:%M'), startdatejob)
        if timezonejob is None and timezonesc is not None:
            if timezonesc != local_timezone:
                start_date = datetime.strptime(startdatejob, '%Y-%m-%d %H:%M')
                comparison = start_date.astimezone(local_timezone)
                self.assertEqual(job_to_run.job['startdate'].strftime('%Y-%m-%d %H:%M'), comparison)
            else:
                self.assertEqual(job_to_run.job['startdate'].strftime('%Y-%m-%d %H:%M'), startdatejob)



    def test_job_invalid_timezone(self):
        """Test that invalid timezone in job raises a ValueError."""
        scheduler = Scheduler(start_date="2024-12-25 10:00", time_zone=0)
        with self.assertRaises(ValueError):
            scheduler.job(start_date="2024-12-25 11:00", time_zone="invalid")

    def test_job_repeat(self):
        """Test that a job can be set to repeat a specified number of times."""
        scheduler = Scheduler(start_date="2024-12-25 10:00", time_zone=0)
        job = scheduler.job(start_date="2024-12-25 11:00", time_zone=0).repeat(5)
        self.assertEqual(job.job['repeats'], 5)

    def test_job_interval(self):
        """Test setting intervals for a job."""
        scheduler = Scheduler(start_date="2024-12-25 10:00", time_zone=0)
        job = scheduler.job(start_date="2024-12-25 11:00", time_zone=0).minute(2)
        self.assertEqual(job.job['interval'], 2)
        self.assertEqual(job.job['unit'], 'minute')

    def test_job_end_date(self):
        """Test setting an end date for a job."""
        timezonesc = 5
        timezonejob = 0
        local_timezone = datetime.now().astimezone().tzinfo

        start_datesc = "2024-12-25 10:00"
        startdatejob = "2024-12-25 11:00"
        enddatejob = "2024-12-26 10:00"
        
        
        scheduler = Scheduler(start_date=start_datesc, time_zone=timezonesc)
        job = scheduler.job(start_date=startdatejob, time_zone=timezonejob).until(enddatejob)
        
        

        if timezonejob:
            if local_timezone != timezonejob:
                end_date = datetime.strptime(enddatejob, '%Y-%m-%d %H:%M')
                comparison = end_date.astimezone(local_timezone)
                self.assertEqual(job.job['end_date'].strftime('%Y-%m-%d %H:%M'), enddatejob)
            else:
                self.assertEqual(job.job['end_date'].strftime('%Y-%m-%d %H:%M'), enddatejob)
        if timezonejob is None and timezonesc is not None:
            if timezonesc != local_timezone:
                end_date = datetime.strptime(enddatejob, '%Y-%m-%d %H:%M')
                comparison = end_date.astimezone(local_timezone)
                self.assertEqual(job.job['end_date'].strftime('%Y-%m-%d %H:%M'), enddatejob)
            else:
                self.assertEqual(job.job['end_date'].strftime('%Y-%m-%d %H:%M'), enddatejob)

    def test_run_all_with_threading(self):
        """Test running all jobs with threading enabled."""
        scheduler = Scheduler(threading=True, start_date="2024-12-25 10:00", time_zone=0)

        def dummy_task():
            print("Task executed")

        job = scheduler.job(start_date="2024-12-25 11:00", time_zone=0).do(dummy_task, name="TestJob")
        scheduler.run_all()
        self.assertIsNotNone(job.job['func'])

    def test_run_all_without_threading(self):
        """Test running all jobs without threading."""
        scheduler = Scheduler(threading=False, start_date="2024-12-25 10:00", time_zone=0)

        def dummy_task():
            print("Task executed")

        job = scheduler.job(start_date="2024-12-25 11:00", time_zone=0).do(dummy_task, name="TestJob")
        scheduler.run_all()
        self.assertIsNotNone(job.job['func'])

    def test_calculate_next_run(self):
        """Test calculation of the next run time for a job."""
        scheduler = Scheduler(start_date="2024-12-25 10:00", time_zone=0)
        job = scheduler.job(start_date="2024-12-25 11:00", time_zone=0).minute(1)
        now = datetime(2024, 12, 25, 11, 0, 0)
        next_run = job.calculate_next_run(now)
        expected_next_run = now + timedelta(minutes=1)
        self.assertEqual(next_run, expected_next_run)

    def test_job_second_interval(self):
        """
        Test creating a job that runs every X seconds.
        We verify that the unit and interval are correctly set 
        and that the next run is calculated properly.
        """
        scheduler = Scheduler(start_date="2024-12-25 10:00", time_zone=0)
        job = (
            scheduler
            .job(start_date="2024-12-25 10:05", time_zone=0)
            .second(5)  # set job to run every 5 seconds
        )

        self.assertEqual(job.job['unit'], 'second')
        self.assertEqual(job.job['interval'], 5)

        # Check next_run calculation
        now = datetime(2024, 12, 25, 10, 5, 0)
        next_run = job.calculate_next_run(now)
        expected_next_run = now + timedelta(seconds=5)
        self.assertEqual(next_run, expected_next_run)

    def test_job_minute_interval(self):
        """
        Test creating a job that runs every X minutes.
        We verify that the unit and interval are correctly set 
        and that the next run is calculated properly.
        """
        scheduler = Scheduler(start_date="2024-12-25 10:00", time_zone=0)
        job = (
            scheduler
            .job(start_date="2024-12-25 10:30", time_zone=0)
            .minute(10)  # runs every 10 minutes
        )

        self.assertEqual(job.job['unit'], 'minute')
        self.assertEqual(job.job['interval'], 10)

        now = datetime(2024, 12, 25, 10, 30, 0)
        next_run = job.calculate_next_run(now)
        expected_next_run = now.replace(second=0) + timedelta(minutes=10)
        self.assertEqual(next_run, expected_next_run)

    def test_job_hour_interval(self):
        """
        Test creating a job that runs every X hours.
        We verify that the unit and interval are correctly set 
        and that the next run is calculated properly.
        """
        scheduler = Scheduler(start_date="2024-12-25 08:00", time_zone=0)
        job = (
            scheduler
            .job(start_date="2024-12-25 09:00", time_zone=0)
            .hour(2)  # runs every 2 hours
        )

        self.assertEqual(job.job['unit'], 'hour')
        self.assertEqual(job.job['interval'], 2)

        now = datetime(2024, 12, 25, 9, 0, 0)
        next_run = job.calculate_next_run(now)
        # The method zeroes out minute/second, so we add 2 hours to the top of the hour
        expected_next_run = now.replace(minute=0, second=0) + timedelta(hours=2)
        self.assertEqual(next_run, expected_next_run)

    def test_job_week_interval(self):
        """
        Test creating a job that runs every X weeks on certain weekdays.
        We verify that the unit, interval, and weekdays are set correctly 
        and that the next run is calculated properly.
        """
        scheduler = Scheduler(start_date="2024-12-20 09:00", time_zone=0)
        # Suppose we want it to run every 1 week on Mondays (weekday=0) at 10:00
        # 'week_day=0' -> Monday. If the test date is also Monday, 
        # we confirm that the next run date gets scheduled accordingly.
        job = (
            scheduler
            .job(start_date="2024-12-23 09:00", time_zone=0)  # 2024-12-23 is a Monday
            .week(every=1, week_day=0, hour="10:00")
        )

        self.assertEqual(job.job['unit'], 'week')
        self.assertEqual(job.job['interval'], 1)
        self.assertIn(0, job.job['week_day'])  # 0 means Monday
        self.assertEqual(job.job['at'], "10:00")

        # Let's check next_run calculation on 2024-12-23 at 09:00
        now = datetime(2024, 12, 23, 9, 0, 0)
        next_run = job.calculate_next_run(now)

        # Since 'hour="10:00"', the next run the same day (2024-12-23) at 10:00 
        # if 'calculate_next_run' logic treats "now" as 09:00.
        expected_next_run = now.replace(hour=10, minute=0, second=0)
        self.assertEqual(next_run, expected_next_run)

    def test_job_week_multiple_days(self):
        """
        Test creating a job that runs every week on multiple weekdays.
        For example, Monday and Wednesday, verifying next_run is the earliest upcoming day.
        """
        scheduler = Scheduler(start_date="2024-12-20 09:00", time_zone=0)
        # Suppose we want it to run every week on Monday (0) and Wednesday (2), at 08:00
        job = (
            scheduler
            .job(start_date="2024-12-23 07:59", time_zone=0)  # 2024-12-23 is Monday
            .week(every=1, week_day=[0, 2], hour="08:00")
        )

        self.assertEqual(job.job['unit'], 'week')
        self.assertEqual(job.job['interval'], 1)
        self.assertIn(0, job.job['week_day'])  # Monday
        self.assertIn(2, job.job['week_day'])  # Wednesday
        self.assertEqual(job.job['at'], "08:00")

        # If current time is Monday 07:59, next_run should be Monday 08:00 (today)
        now = datetime(2024, 12, 23, 7, 59, 0)
        next_run = job.calculate_next_run(now)
        expected_next_run = now.replace(hour=8, minute=0, second=0)
        self.assertEqual(next_run, expected_next_run)

    def test_job_with_timezones_second(self):
        """
        Test that a job set to run every X seconds honors different time zones
        by checking how the start date is converted to local time.
        """
        # Let's pick a scheduler in UTC+2
        scheduler = Scheduler(start_date="2024-12-25 10:00", time_zone=2)

        # Now create a job that starts in UTC-1
        job = scheduler.job(start_date="2024-12-25 10:00", time_zone=-1).second(15)

        self.assertEqual(job.job['unit'], 'second')
        self.assertEqual(job.job['interval'], 15)

        # Check that the job's start_date is converted properly to local time
        # according to the system's local timezone. 
        # (We won't do a hard-coded assert here because each environment can differ.
        #  Instead, we ensure it doesn't raise any exceptions and that the tzinfo is set.)
        self.assertIsNotNone(job.job['startdate'].tzinfo, "Job startdate timezone should not be None")

        # We can still do a sanity check: the job's 'time_zone' is UTC-1
        self.assertEqual(job.job['time_zone'].utcoffset(None), timedelta(hours=-1))

        # Confirm next run calculation from a reference now
        now_local = datetime(2024, 12, 25, 10, 0, 0)  # local time
        next_run = job.calculate_next_run(now_local)
        expected_next_run = now_local + timedelta(seconds=15)
        self.assertEqual(next_run, expected_next_run)

    def test_job_with_timezones_hour(self):
        """
        Test that a job set to run every X hours works properly with different time zones.
        """
        # Scheduler in UTC+5
        scheduler = Scheduler(start_date="2024-12-25 00:00", time_zone=5)

        # Create a job that starts in UTC-3
        job = scheduler.job(start_date="2024-12-25 05:00", time_zone=-3).hour(1)

        self.assertEqual(job.job['unit'], 'hour')
        self.assertEqual(job.job['interval'], 1)

        # Check that the job's start_date is converted properly to local time
        self.assertIsNotNone(job.job['startdate'].tzinfo)

        # Confirm next run calculation from a reference now
        # We pretend "now" is local system time; we just want to see if
        # the job calculates the next run in 1 hour from that reference.
        now_local = datetime(2024, 12, 25, 5, 0, 0)
        next_run = job.calculate_next_run(now_local)
        expected_next_run = now_local.replace(minute=0, second=0) + timedelta(hours=1)
        self.assertEqual(next_run, expected_next_run)

if __name__ == "__main__":
    unittest.main(argv=[''], exit=False, verbosity=2)


# 


test_calculate_next_run (__main__.TestScheduler.test_calculate_next_run)
Test calculation of the next run time for a job. ... ok
test_invalid_start_date (__main__.TestScheduler.test_invalid_start_date)
Test that invalid start_date raises a ValueError. ... ok
test_job_creation (__main__.TestScheduler.test_job_creation)
Test creating a job within the scheduler. ... ok
test_job_end_date (__main__.TestScheduler.test_job_end_date)
Test setting an end date for a job. ... ok
test_job_hour_interval (__main__.TestScheduler.test_job_hour_interval)
Test creating a job that runs every X hours. ... ok
test_job_interval (__main__.TestScheduler.test_job_interval)
Test setting intervals for a job. ... ok
test_job_invalid_timezone (__main__.TestScheduler.test_job_invalid_timezone)
Test that invalid timezone in job raises a ValueError. ... ok
test_job_minute_interval (__main__.TestScheduler.test_job_minute_interval)
Test creating a job that runs every X minutes. ... ok
test_job_repeat (__main__.TestSche

this is scheduler startdate and timezone inside scheduler class:
 UTC
2024-12-25 14:00:00+04:00

this is job timezone UTC
this is job startdate and timezone inside job function: 
UTC
2024-12-25 15:00:00+04:00

calc next run: 2024-12-25 11:01:00
this is scheduler startdate and timezone inside scheduler class:
 UTC+05:00
2024-12-25 09:00:00+04:00

this is job timezone UTC
this is job startdate and timezone inside job function: 
UTC
2024-12-25 15:00:00+04:00

this is scheduler startdate and timezone inside scheduler class:
 UTC+05:00
2024-12-25 09:00:00+04:00

this is job timezone UTC
this is job startdate and timezone inside job function: 
UTC
2024-12-25 15:00:00+04:00

This is the job's timezone and the converted end date in 'until':
UTC
2024-12-26 14:00:00+04:00

this is job timezone abd end date in untill:
 UTC
 2024-12-26 10:00:00

this is scheduler startdate and timezone inside scheduler class:
 UTC
2024-12-25 12:00:00+04:00

this is job timezone UTC
this is job startdate and timezo